In [2]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Configure app with proper asset path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🦎 Reptile Carousel"

# 📂 Load reptile species
REPTILE_DIR = os.path.abspath("../assets/reptiles")
REPTILE_SPECIES = sorted([
    d for d in os.listdir(REPTILE_DIR)
    if os.path.isdir(os.path.join(REPTILE_DIR, d))
])

# 📸 Dictionary of {species: [image paths]}
reptile_images = {}
for species in REPTILE_SPECIES:
    folder = os.path.join(REPTILE_DIR, species)
    images = []
    for root, _, files in os.walk(folder):
        rel_root = os.path.relpath(root, os.path.abspath("../assets"))
        images += [f"/assets/{rel_root}/{f}" for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    reptile_images[species] = sorted(images)

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🦎 Reptile Carousel", style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div([
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.replace("_", " ").capitalize(), "value": s} for s in REPTILE_SPECIES],
            value=REPTILE_SPECIES[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Display carousel content based on species
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = reptile_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginRight": "20px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic"})
    ])

# 🔁 Handle navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run app
clear_output(wait=True)
app.run(port=8169)
